In [21]:
%load_ext autoreload
%autoreload 1
%aimport ecg_get_data
%aimport Models
%aimport train_test_validat
%aimport self_attention
%aimport ECGplot
import Models 
from train_test_validat import *
from self_attention import *
from  ecg_get_data import *
import neurokit2
import matplotlib.pyplot as plt

import torch
import torch.utils.data as Data
import random

import time
random_seed = 1
torch.manual_seed(random_seed)    # reproducible
torch.cuda.manual_seed_all(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime()) 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:

data_path =  '/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/npy_ECG/' #路径
lable_path = '/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/label.npy'
model_path = '/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/model/'+time_str
os.makedirs(model_path, exist_ok=True)
log_path = '/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/log/'+  time_str


EcgChannles_num = 12
EcgLength_num = 5000

In [6]:
x = load_data(data_path,EcgChannles_num=EcgChannles_num,EcgLength_num=EcgLength_num)
y = load_label(lable_path)

100%|██████████| 1698/1698 [00:10<00:00, 166.36it/s]


In [7]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(x,y,train_size=0.9,random_state = random_seed,shuffle = True,stratify=y)
print("         HTN     NHTN ")
print("train: %5d   %5d" % (train_y.sum(),len(train_y)-train_y.sum()))
print("test : %5d   %5d" % (test_y.sum(),len(test_y)-test_y.sum()))

         HTN     NHTN 
train:   628     900
test :    70     100


In [5]:
#train_dataset,valid_dataset = load_numpy_dataset_to_tensor_dataset(x,y,random_seed=random_seed,train_rate = 0.8)

In [8]:
BATCH_SIZE = 128
FOLDS = 1
EPOCHS = 2000  
PATIENCE = 64
LR = 0.00001
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [7]:
from torch.utils.tensorboard import SummaryWriter   
writer = SummaryWriter(log_path)
from torchsummary import summary

In [8]:
NET = [Models.CNN_ATT3() for i in range(FOLDS)]
print(summary(NET[0], (EcgChannles_num,EcgLength_num), device="cpu"))
writer.add_graph(NET[0],torch.rand([1,EcgChannles_num,EcgLength_num]))

0.0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1             [-1, 12, 5000]              24
            Conv1d-2             [-1, 32, 5000]           4,256
              ReLU-3             [-1, 32, 5000]               0
       BatchNorm1d-4             [-1, 32, 5000]              64
            Conv1d-5             [-1, 64, 5000]          22,592
              ReLU-6             [-1, 64, 5000]               0
         MaxPool1d-7             [-1, 64, 1250]               0
       BatchNorm1d-8             [-1, 64, 1250]             128
            Conv1d-9            [-1, 128, 1250]          24,704
             ReLU-10            [-1, 128, 1250]               0
      BatchNorm1d-11            [-1, 128, 1250]             256
           Conv1d-12            [-1, 256, 1250]          98,560
             ReLU-13            [-1, 256, 1250]               0
        MaxPool1d-14             [-

/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/self_attention.py:66: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if dim % 2 != 0:
/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/self_attention.py:70: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  i_matrix = torch.arange(dim//2, dtype=torch.float)
/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/self_attention.py:85: TracerWarning: Converti

0.0
0.0


In [9]:
from torch.optim.lr_scheduler import CosineAnnealingLR

for fold in range(FOLDS):
    
    early_stopping = EarlyStopping(PATIENCE, verbose=True, model_path=model_path, delta=0)
    train_dataset,valid_dataset = get_k_fold_dataset(fold=int(fold+1),x = train_x,y=train_y,k=FOLDS,random_seed = random_seed)
    train_dataloader = Data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    valid_dataloader = Data.DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

    NET[fold].to(DEVICE)
    optimizer  = torch.optim.Adam(NET[fold].parameters(), lr=LR)  
    criterion = torch.nn.CrossEntropyLoss()   
    #等间隔调整学习率
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max = 32)
    best_loss = 3
    for epoch in range(1,EPOCHS):
        time_all=0
        start_time = time.time()
        train_loss,train_acc = train_model(train_dataloader, NET[fold], criterion, optimizer,DEVICE) # 训练模型
        #scheduler.step() # 学习率迭代

        time_all = time.time()-start_time
        validate_loss,validate_acc = test_model(train_dataloader,criterion,NET[fold],DEVICE) # 测试模型
        writer.add_scalars(main_tag=str(fold)+'_Loss',tag_scalar_dict={'train': train_loss,'validate': validate_loss},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_Accuracy',tag_scalar_dict={'train': train_acc,'validate': validate_acc},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_LearningRate',tag_scalar_dict={'LR': optimizer.state_dict()['param_groups'][0]['lr']},global_step=epoch)

        print('- Epoch: %d - Train_loss: %.5f - Train_acc: %.5f - Val_loss: %.5f - Val_acc: %5f - T_Time: %.3f' %(epoch,train_loss,train_acc,validate_loss,validate_acc,time_all))
        print('当前学习率：%f' %optimizer.state_dict()['param_groups'][0]['lr'])

        if validate_loss < best_loss:
            best_loss = validate_loss
            print('Find better model in Epoch {0}, saving model.'.format(epoch))
            torch.save(NET[fold],  model_path+'/best_model_' + str(fold) + '.pt')  # 保存最优模型
        #是否满足早停法条件
        if(early_stopping(validate_loss,NET[fold])):
            print("Early stopping")
            break

    print('Fold %d Training Finished' %(fold+1))
    torch.cuda.empty_cache()# 清空显卡cuda
print('Training Finished')

1528
               HTN  NHTN
valid_dataset: 135  171
train_dataset: 493  1087
0.0
9.9999e-06
1.0418e-05
1.6129e-05
2.3584e-05
2.6977e-05
2.803e-05
2.7802e-05
3.0598e-05
3.3928e-05
3.9572e-05
3.9572e-05
3.9572e-05
3.9572e-05
3.9572e-05
3.9572e-05
3.9572e-05
3.9572e-05
3.9572e-05
3.9572e-05
- Epoch: 1 - Train_loss: 0.67753 - Train_acc: 0.57420 - Val_loss: 0.67334 - Val_acc: 0.598013 - T_Time: 5.712
当前学习率：0.000010
Find better model in Epoch 1, saving model.
Validation loss decreased (inf --> 0.673336).  Saving model ...
                    --------------------------------------------------

3.9572e-05
4.4127e-05
4.2965e-05
4.319e-05
4.6813e-05
4.8767e-05
5.1026e-05
5.5104e-05
6.06e-05
6.6306e-05
7.2892e-05
7.2892e-05
7.2892e-05
7.2892e-05
7.2892e-05
7.2892e-05
7.2892e-05
7.2892e-05
7.2892e-05
7.2892e-05
- Epoch: 2 - Train_loss: 0.65390 - Train_acc: 0.60871 - Val_loss: 0.67585 - Val_acc: 0.608013 - T_Time: 1.423
当前学习率：0.000010
EarlyStopping counter: 1 out of 64

7.2892e-05
8.034e-05
8.767

# ------Display the attention value-----

# -----For Leads -----

In [ ]:
test_model_path = "/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN/model/20220907_091538/best_model_0.pt"
test_model = torch.load(test_model_path).to(DEVICE)
test_x = x[0:1698]
test_y = x[0:1698]
test_x = MAX_MIN_normalization_by_feactures(test_x)
test_x = torch.FloatTensor(test_x)  #turn numpy to tensor
test_y = torch.LongTensor(test_y)
test_dataset = Data.TensorDataset(test_x, test_y)
test_dataloader = Data.DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)
test_model.eval()

test_model.eval()
attention_value_timestep = np.zeros(EcgChannles_num,)
for i,data in enumerate(test_dataloader,0):
    inputs,labels = data[0].to(DEVICE),data[1].to(DEVICE)
    outputs = test_model(inputs)
    #print(outputs)
    _,pred = outputs.max(1) # 求概率最大值对应的标签
    #print("the label :{labels},pred is {pred}".format(labels=labels[0],pred=pred[0]))
    attention_value = test_model.attention_value
    attention_value_timestep += (((attention_value.to('cpu'))[0]).detach().numpy()).sum(axis=0)
plt.figure(figsize=(5,5))
plt.stem(np.arange(0,len(attention_value_timestep)), attention_value_timestep)
plt.xticks(np.arange(0,len(attention_value_timestep)),['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])
plt.ylabel("Sum of attention value")
plt.show()